In [1]:
%%capture
%load_ext autoreload
%autoreload 2
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as pl
from prfpy.stimulus import PRFStimulus2D
from prfpy.model import Iso2DGaussianModel, Norm_Iso2DGaussianModel, DoG_Iso2DGaussianModel, CSS_Iso2DGaussianModel
from prfpy.fit import Iso2DGaussianFitter, Norm_Iso2DGaussianFitter, DoG_Iso2DGaussianFitter, CSS_Iso2DGaussianFitter

This notebook shows a simple example of prfpy use

# Creating stimulus object

In [3]:
screen_size_cm=40
screen_distance_cm=200
design_matrix=np.zeros((50,50,100))
design_matrix[:,20:30,40:50] = 1
TR=1
task_lengths=[100]
task_names=['test']
late_iso_dict={'test':np.arange(40)}
normalize_integral_dx=False

prf_stim = PRFStimulus2D(screen_size_cm=screen_size_cm,
                             screen_distance_cm=screen_distance_cm,
                             design_matrix=design_matrix,
                             TR=TR,
                             task_lengths=task_lengths,
                             task_names=task_names,
                             late_iso_dict=late_iso_dict,
                             normalize_integral_dx=normalize_integral_dx)

# Gaussian model fit

## Creating Gaussian model and fitter objects

In [4]:
#only needed if filtering predictions
filter_type='dc'
filter_params={"first_modes_to_remove":3,
                         "last_modes_to_remove_percent":0,
                         "window_length":50,
                         "polyorder":3,
                         "highpass":True,
                         "add_mean":True}

filter_predictions=False

hrf=[1,4,0]

data = 3*np.random.rand(50,100)+0.01*np.sum(design_matrix, axis=(0,1))-1.5
np.roll(data,6)

normalize_RFs=False


gg = Iso2DGaussianModel(stimulus=prf_stim,
                          hrf=hrf,
                          filter_predictions=filter_predictions,
                          filter_type=filter_type,
                          filter_params=filter_params,
                          normalize_RFs=normalize_RFs)

gf = Iso2DGaussianFitter(data=data, model=gg, n_jobs=8)

## Gaussian grid fit

In [5]:
ecc_grid=np.linspace(0,10,10)
polar_grid=np.linspace(-np.pi,np.pi,10)
size_grid=np.linspace(1,10,10)
verbose=False
n_batches=8
fixed_grid_baseline=0
gauss_grid_bounds=[(0,1000)] #bound on prf amplitudes (only positive)

hrf_1_grid=np.linspace(0,10,10)
hrf_2_grid=np.linspace(0,0,1)

gf.grid_fit(ecc_grid=ecc_grid,
                polar_grid=polar_grid,
                size_grid=size_grid,
                verbose=verbose,
                n_batches=n_batches,
                fixed_grid_baseline=fixed_grid_baseline,
                grid_bounds=gauss_grid_bounds,
           hrf_1_grid=hrf_1_grid,
           hrf_2_grid=hrf_2_grid)

/Users/marcoaqil/prfpy/prfpy/model.py:314: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':


In [ ]:
gf.gridsearch_params

## Gaussian Iterative Fit

In [6]:
rsq_threshold=-10
verbose=True
gauss_bounds = [(-1.5*10, 1.5*10),  # x
                (-1.5*10, 1.5*10),  # y
                (0.1, 1.5*5),  # prf size
                (0, 1000),  # prf amplitude
                (0, 0)]  # bold baseline
gauss_bounds += [(0,10),(0,0)] #hrf bounds. if want it fixed to some value, specify e.g. (4,4) (0,0)
constraints=None
xtol=1e-4
ftol=1e-4

gf.iterative_fit(rsq_threshold=rsq_threshold, verbose=verbose,
                         bounds=gauss_bounds,
                         constraints=constraints,
                             xtol=xtol,
                             ftol=xtol)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
/Users/marcoaqil/anaconda3/envs/prfpy_2021/lib/python3.9/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '
/Users/marcoaqil/anaconda3/envs/prfpy_2021/lib/python3.9/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '
/Users/marcoaqil/anaconda3/envs/prfpy_2021/lib/python3.9/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '
/Users/marcoaqil/anaconda3/envs/prfpy_2021/lib/python3.9/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change i

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.96093D+02    |proj g|=  2.32902D-01
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.74185D+02    |proj g|=  2.94673D-01
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.51767D+02    |proj g|=  4.76609D-01
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.74282D+02    |proj g|=  2.35934D-01
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =     

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.14866D+02    |proj g|=  3.39659D-01

At iterate    1    f=  2.49288D+02    |proj g|=  2.34712D+01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5      1      3      2     0     1   2.347D+01   2.493D+02
  F =   249.28839344988009     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             

At iterate    1    f=  1.79767D+02    |proj g|=  5.80661D+01


           * * *

At iterate    1    f=  1.86753D+02    |proj g|=  2


At iterate    1    f=  2.34055D+02    |proj g|=  1.28318D+00

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5      1      3      1     0     0   1.283D+00   2.341D+02
  F =   234.05517155395864     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             

At iterate    1    f=  2.42842D+02    |proj g|=  8.38781D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final proj

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.38203D+02    |proj g|=  4.23609D-01
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.16208D+02    |proj g|=  6.84679D-02
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.09904D+02    |proj g|=  4.55566D-01
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.09488D+02    |proj g|=  7.85946D-02
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =     

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    2.6s


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.92701D+02    |proj g|=  5.56670D-01
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         1 variables are exactly at the bounds

At iterate    0    f=  2.44873D+02    |proj g|=  9.48319D-02
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.94988D+02    |proj g|=  1.09623D-01
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         1 variables are exactly at the bounds

At iterate    0    f=  2.24758D+02    |proj g|=  6.36522D+02

At iterate    1    f=  2.33471D+02    |proj g|=  1.45113D+02
RUNNING THE L-BFGS-B C

[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    2.9s finished


In [7]:
gf.iterative_search_params

array([[ 5.69676861e-07,  1.70903058e-06,  1.00001652e+00,
         2.45583813e-02,  0.00000000e+00,  7.73048776e+00,
         0.00000000e+00,  4.22432499e-01],
       [ 0.00000000e+00,  0.00000000e+00,  9.99998092e-01,
         2.29528738e-02,  0.00000000e+00,  5.57534051e+00,
         0.00000000e+00,  3.26755657e-01],
       [ 7.30872039e+00, -2.66015667e+00,  5.00000000e+00,
         1.21256878e-02,  0.00000000e+00,  8.88903795e+00,
         0.00000000e+00,  2.97539250e-01],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         2.15103470e-02,  0.00000000e+00,  6.63969425e+00,
         0.00000000e+00,  3.62163095e-01],
       [-1.48417477e-06,  0.00000000e+00,  1.00000891e+00,
         2.07455954e-02,  0.00000000e+00,  7.85491925e+00,
         0.00000000e+00,  3.10452253e-01],
       [-1.10550756e-07,  1.10550756e-07,  1.00000166e+00,
         2.24531659e-02,  0.00000000e+00,  7.76871870e+00,
         0.00000000e+00,  3.39430671e-01],
       [-2.14045111e-07,  2.140451

# DN model fit

## Create model and fitter objects

In [8]:
stimulus=prf_stim
hrf=[1,5,0]
filter_type='dc'
filter_params={"first_modes_to_remove":3,
                         "last_modes_to_remove_percent":0,
                         "window_length":50,
                         "polyorder":3,
                         "highpass":True,
                         "add_mean":True}

filter_predictions=False                                     
normalize_RFs=False

use_previous_gaussian_fitter_hrf=True #if true, will use hrf result from gauss fit at the grid stage instead of doing a grid fit for it

gg_norm = Norm_Iso2DGaussianModel(stimulus=prf_stim,
                                    hrf=hrf,
                                    filter_predictions=filter_predictions,
                                    filter_type=filter_type,
                                    filter_params=filter_params,                                       
                                    normalize_RFs=normalize_RFs)

gf_norm = Norm_Iso2DGaussianFitter(data=data,
                                   model=gg_norm,
                                   n_jobs=8,
                                   previous_gaussian_fitter=gf,
                                  use_previous_gaussian_fitter_hrf=use_previous_gaussian_fitter_hrf) 

## DN model grid fit

In [9]:
norm_grid_bounds = [(0,1000),(0,1000)] #only prf amplitudes between 0 and 1000, only neural baseline values between 0 and 1000

gf_norm.grid_fit(surround_amplitude_grid=np.linspace(0,10,5),
             surround_size_grid=np.linspace(1,10,5),
             neural_baseline_grid=np.linspace(0,10,5),
             surround_baseline_grid=np.linspace(1,10,5),
             verbose=verbose,
             n_batches=8,
             rsq_threshold=rsq_threshold,
             fixed_grid_baseline=fixed_grid_baseline,
             grid_bounds=norm_grid_bounds,
             hrf_1_grid=np.linspace(0,10,5),
             hrf_2_grid=np.linspace(0,0,1))

Using HRF from previous gaussian iterative fit
Each batch contains approx. 7 voxels.


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
/Users/marcoaqil/prfpy/prfpy/model.py:545: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':
/Users/marcoaqil/prfpy/prfpy/model.py:545: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':
/Users/marcoaqil/prfpy/prfpy/model.py:545: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':
/Users/marcoaqil/prfpy/prfpy/model.py:545: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':
/Users/marcoaqil/prfpy/prfpy/model.py:545: FutureWarning: elementwise comparison failed; returning scalar instead, but in the fut

## DN Iterative Fit

In [17]:
norm_bounds =  [(-1.5*10, 1.5*10),  # x
                (-1.5*10, 1.5*10),  # y
                (0.1, 1.5*5),  # prf size
                (0, 1000),  # prf amplitude
                (0, 0),  # bold baseline
                (0, 1000),  # surround amplitude
                (0.1, 3*5),  # surround size
                (0, 1000),  # neural baseline
                (1e-6, 1000)]  # surround baseline
norm_bounds += [(0,10),(0,0)]
constraints_norm = []

gf_norm.iterative_fit(rsq_threshold=rsq_threshold, verbose=verbose,
                               bounds=norm_bounds,
                               constraints=constraints_norm,
                               xtol=xtol,
                               ftol=ftol)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
/Users/marcoaqil/anaconda3/envs/prfpy_2021/lib/python3.9/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '
/Users/marcoaqil/anaconda3/envs/prfpy_2021/lib/python3.9/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '
/Users/marcoaqil/anaconda3/envs/prfpy_2021/lib/python3.9/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. 

# Crossvalidate

In [20]:
test_data = 3*np.random.rand(50,100)+0.01*np.sum(design_matrix, axis=(0,1))-1.5
np.roll(test_data,6)

test_stimulus = prf_stim

single_hrf = False

gf.crossvalidate_fit(test_data=test_data,
                     test_stimulus=test_stimulus,
                     single_hrf=single_hrf)

gf_norm.crossvalidate_fit(test_data,
                        test_stimulus=test_stimulus,
                        single_hrf=single_hrf)

/Users/marcoaqil/prfpy/prfpy/model.py:314: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':
/Users/marcoaqil/prfpy/prfpy/model.py:545: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':
